In [1]:
import os
import io
import pandas as pd
from pandas import DataFrame
import sys
import numpy as np
import json
import pickle
import jsonpickle
import time
import random

import google.cloud
from google.cloud import datastore
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from sqlalchemy import *
import sqlalchemy
import sqlalchemy.schema
from sqlalchemy.types import *

from IPython.display import *
from pandas.io.json import *
# gcloud auth application-default login

In [2]:
#df = pd.read_pickle('/home/jm/Documents/testingStuff/my_file.pkl')
engine = create_engine('mysql+pymysql://root:@localhost/sawi_tweets?charset=utf8mb4', encoding='utf8', echo = False)
df = pd.read_sql_table("sawi_tweets_historical", con = engine)

/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_1` CHECK (`user_verified` in (0,1)),'
  util.warn("Unknown schema content: %r" % line)
/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_2` CHECK (`RT` in (0,1))'
  util.warn("Unknown schema content: %r" % line)


In [3]:
df.head()

,index,created_at,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_count,screen_name,text,tweet_id,...,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
0,0,2017-09-24 23:54:27,2,None,None,None,0,AmanteTech,Would next #Apple #iOS update cause your exist...,912102964881367040,...,2133,635,632,869739471570325504,"Sydney, New South Wales",AmanteTech,6314,Pacific Time (US & Canada),0,0
1,1,2017-09-24 23:30:33,0,None,None,None,84,supermeg500,RT @MegsSkunkey: iPhone 8 giveaway!!\nhttps://...,912096951532847104,...,61,361,4512,819370348420284417,,Giveaways~,402,None,0,1
2,2,2017-09-24 22:45:06,0,None,None,None,1860,botlamaya,RT @Sudathchamin: iPhone X new price?Ur left k...,912085514282553344,...,10,617,274,941255144,Colombo,Bot Lamaya [beta],51626,New Delhi,0,1
3,3,2017-09-24 22:45:06,0,None,None,None,1860,hikma_5000,RT @Sudathchamin: iPhone X new price?Ur left k...,912085512776798213,...,1388,143,113,791115735200182272,"Mansfield, TX",Hikma🌹🌹🌹,5432,Pacific Time (US & Canada),0,1
4,4,2017-09-24 22:36:56,0,None,None,None,4,Joey2031,RT @BoinkHunter: Beautiful device. \n#iPhoneX ...,912083456552783873,...,853,66,75,705859071044423680,,Joey,646,None,0,1


In [4]:
df.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,22364.000000,22364.000000,22364.000000,2.236400e+04,22364.000000,2.236400e+04,22364.000000,2.236400e+04,2.236400e+04,22364.000000,22364.000000
mean,48.985557,0.458371,4310.406993,9.146561e+17,8501.743427,7.903254e+03,1812.930290,1.891122e+17,4.485516e+04,0.012341,0.549097
std,28.843383,9.504987,10492.972405,9.111286e+14,23193.527125,7.776805e+04,11221.314492,3.475826e+17,1.277739e+05,0.110406,0.497595
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000e+00,0.000000,2.172000e+03,1.000000e+00,0.000000,0.000000
25%,24.000000,0.000000,0.000000,9.142210e+17,101.000000,9.700000e+01,132.000000,3.175787e+08,1.412000e+03,0.000000,0.000000
50%,49.000000,0.000000,4.000000,9.153297e+17,1278.000000,3.730000e+02,351.000000,1.869125e+09,8.120000e+03,0.000000,1.000000
75%,74.000000,0.000000,2519.000000,9.153474e+17,7147.000000,1.299250e+03,956.000000,4.450913e+09,3.455425e+04,0.000000,1.000000
max,99.000000,1263.000000,142939.000000,9.153658e+17,589799.000000,4.294144e+06,851643.000000,9.153586e+17,2.210810e+06,1.000000,1.000000


In [5]:
df.count()

index                      22364
created_at                 22364
favorite_count             22364
in_reply_to_screen_name     1732
in_reply_to_status_id       1413
in_reply_to_user_id         1732
retweet_count              22364
screen_name                22364
text                       22364
tweet_id                   22364
tweet_location                44
user_created_at            22364
user_description           22364
user_favorites_count       22364
user_followers_count       22364
user_friends_count         22364
user_id                    22364
user_location              22364
user_name                  22364
user_statuses_count        22364
user_time_zone             13316
user_verified              22364
RT                         22364
dtype: int64

In [6]:
dfs = df.drop_duplicates(subset = 'tweet_id', inplace=False)

In [7]:
dfs.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,21778.000000,21778.000000,21778.000000,2.177800e+04,21778.000000,2.177800e+04,21778.000000,2.177800e+04,2.177800e+04,21778.000000,21778.000000
mean,49.072688,0.462531,4425.200753,9.147164e+17,8665.691615,8.035060e+03,1818.128478,1.836138e+17,4.539588e+04,0.012306,0.548903
std,28.839817,9.629868,10609.520958,8.381596e+14,23428.676374,7.878039e+04,11332.575125,3.433107e+17,1.279264e+05,0.110250,0.497614
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000e+00,0.000000,2.172000e+03,1.000000e+00,0.000000,0.000000
25%,24.000000,0.000000,0.000000,9.142230e+17,112.250000,1.020000e+02,135.000000,3.179577e+08,1.487000e+03,0.000000,0.000000
50%,49.000000,0.000000,4.000000,9.153308e+17,1335.000000,3.780000e+02,356.000000,1.730809e+09,8.477000e+03,0.000000,1.000000
75%,74.000000,0.000000,2521.000000,9.153479e+17,7362.750000,1.299000e+03,958.000000,4.236847e+09,3.544475e+04,0.000000,1.000000
max,99.000000,1263.000000,142939.000000,9.153658e+17,589799.000000,4.294144e+06,851643.000000,9.153586e+17,2.210810e+06,1.000000,1.000000


In [8]:
dfs.count()

index                      21778
created_at                 21778
favorite_count             21778
in_reply_to_screen_name     1732
in_reply_to_status_id       1413
in_reply_to_user_id         1732
retweet_count              21778
screen_name                21778
text                       21778
tweet_id                   21778
tweet_location                39
user_created_at            21778
user_description           21778
user_favorites_count       21778
user_followers_count       21778
user_friends_count         21778
user_id                    21778
user_location              21778
user_name                  21778
user_statuses_count        21778
user_time_zone             13027
user_verified              21778
RT                         21778
dtype: int64

In [9]:
dfs.to_csv("twitter_dataset_v1.csv")